DEVOIR 5 : ANALYSE SYNTAXIQUE DE COMPTES-RENDUS MÉDICAUX

Encadré par : Pr. Ikram Benabdelouahab
  Réalisé par : Aya Khalifi



In [1]:
import spacy
from spacy import displacy
import re

# Charger modèle français
nlp = spacy.load("fr_core_news_sm")

# Partie 1 : POS Tagging Médical

def tokenize_medical(text):
    """
    Fonction pour traiter les tokens médicaux mal segmentés (ex: '3x/jour').
    Remplace les occurrences par une forme standard.
    """
    # Remplacement simple : met en forme '3x/jour' → '3x_par_jour'
    return re.sub(r"(\d+)x/jour", r"\1x_par_jour", text)


def pos_tagging(text):
    """
    Annotation grammaticale complète : texte, POS, lemme, dépendance.
    """
    text = tokenize_medical(text)
    doc = nlp(text)
    for token in doc:
        print(f"{token.text:<15} POS: {token.pos_:<6} Lemma: {token.lemma_:<12} Dep: {token.dep_}")
    return doc


def extract_symptoms(doc):
    """
    Extraction des symptômes : noms + adjectifs associés.
    Exemple : 'toux persistante'.
    """
    symptoms = []
    for token in doc:
        if token.pos_ in ("NOUN", "PROPN") and any(child.dep_ == "amod" for child in token.children):
            # nom avec adjectif
            adj = [child.text for child in token.children if child.dep_ == "amod"]
            symptoms.append(f"{token.text} {' '.join(adj)}")
    return symptoms


def extract_treatments(doc):
    """
    Extraction des traitements : verbe 'prescrire' et noms de médicaments.
    """
    treatments = []
    # trouve verbes 'prescr' et objets
    for token in doc:
        if token.lemma_ == "prescrire" and token.pos_ == "VERB":
            objs = [child.text for child in token.rights if child.dep_ in ("obj", "dobj")]
            treatments.extend(objs)
    return treatments

# Partie 2 : Extraction de Relations

def extraire_relations(text):
    """
    Extrait les triplets sujet-verbe-objet et motifs de raison.
    """
    text_proc = tokenize_medical(text)
    doc = nlp(text_proc)
    relations = []
    for token in doc:
        if token.pos_ == "VERB":
            subj = [t.text for t in token.lefts if t.dep_ == "nsubj"]
            obj = [t.text for t in token.rights if t.dep_ in ("obj", "dobj")]
            if subj and obj:
                relations.append((subj[0], token.lemma_, obj[0]))
        # motif de raison: structure 'raison' dépendant d'un nom
        if token.lemma_ == "raison":
            head = token.head
            relations.append((token.text, "raison", head.lemma_))
    return relations


def detect_negations(text):
    """
    Détecte négations et marque les relations.
    Ex: 'ne pas prescrire'.
    """
    text_proc = tokenize_medical(text)
    doc = nlp(text_proc)
    neg_rels = []
    for token in doc:
        if token.dep_ == "neg":
            # trouver verbe parent
            verb = token.head
            subj = [t.text for t in verb.lefts if t.dep_ == "nsubj"]
            objs = [t.text for t in verb.rights if t.dep_ in ("obj", "dobj")]
            if subj and objs:
                neg_rels.append((subj[0], f"NE PAS {verb.lemma_}", objs[0]))
    return neg_rels

# Partie 3 : Analyse de Dépendances

def visualize_dependencies(text, jupyter=False):
    """
    Affiche un graphe de dépendances via displacy.
    Si jupyter=True, renvoie le rendu inline.
    """
    text_proc = tokenize_medical(text)
    doc = nlp(text_proc)
    svg = displacy.render(doc, style="dep", jupyter=jupyter)
    return svg


def extract_dosage(text):
    """
    Trouve tous les médicaments associés à une posologie.
    Ex: ('amoxicilline', '500mg 3x/jour')
    """
    text_proc = tokenize_medical(text)
    doc = nlp(text_proc)
    meds = []
    # regex pour dosage
    dosage_pattern = re.compile(r"(\d+mg(?:_par_jour| \d+x_par_jour)?)")
    for token in doc:
        if token.pos_ == "NOUN":
            # cherche dosage après nom
            span = text_proc[token.idx:]
            match = dosage_pattern.search(span)
            if match:
                meds.append((token.text, match.group(1).replace("_", " ")))
    return meds

# Exemple de tests
if __name__ == "__main__":
    # P1
    doc1 = pos_tagging("La patiente âgée de 65 ans présente une toux persistante et une fièvre à 38.5°C. Le médecin prescrit de l'amoxicilline 500mg 3x/jour.")
    print("Symptômes extraits:", extract_symptoms(doc1))
    print("Traitements extraits:", extract_treatments(doc1))

    # P2
    print("Relations:", extraire_relations("Le médecin arrête l'aspirine en raison de saignements."))
    print("Négations:", detect_negations("Le médecin ne pas prescrire le paracétamol."))

    # P3
    svg = visualize_dependencies("Après analyse, le cardiologue recommande un scanner cardiaque immédiat.")
    with open("dep_graphe.svg", "w", encoding="utf-8") as f:
        f.write(svg)
    print("Graphe de dépendances sauvegardé: dep_graphe.svg")

    # P3.2
    print("Dosages:", extract_dosage("amoxicilline 500mg 3x/jour ibuprofène 400mg"))


La              POS: DET    Lemma: le           Dep: det
patiente        POS: NOUN   Lemma: patient      Dep: nsubj
âgée            POS: VERB   Lemma: âger         Dep: acl
de              POS: ADP    Lemma: de           Dep: case
65              POS: NUM    Lemma: 65           Dep: nummod
ans             POS: NOUN   Lemma: an           Dep: obl:arg
présente        POS: VERB   Lemma: présente     Dep: ROOT
une             POS: DET    Lemma: un           Dep: det
toux            POS: ADP    Lemma: toux         Dep: case
persistante     POS: NOUN   Lemma: persistante  Dep: obj
et              POS: CCONJ  Lemma: et           Dep: cc
une             POS: DET    Lemma: un           Dep: det
fièvre          POS: NOUN   Lemma: fièvre       Dep: conj
à               POS: ADP    Lemma: à            Dep: case
38.5            POS: NOUN   Lemma: 38.5         Dep: nmod
°               POS: NOUN   Lemma: degré        Dep: nmod
C               POS: NOUN   Lemma: c            Dep: nmod
.              